In [1]:
!pip install geopandas shapely fiona pyproj


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:

input_folder = "/content/drive/My Drive/Shapefiles"  # Folder containing input vector files # @param {type:"string"}
output_folder = "/content/drive/My Drive/Shapefile_MBB"  # Folder to save oriented bounding rectangles # @param {type:"string"}

import os
os.makedirs(output_folder, exist_ok=True)

print(f"Input folder: {input_folder}")
print(f"Output folder: {output_folder}")


Input folder: /content/drive/My Drive/Shapefiles
Output folder: /content/drive/My Drive/Shapefile_OMB


In [4]:
import glob
import geopandas as gpd
from shapely.geometry import Polygon

# Supported vector extensions
extensions = ['*.shp', '*.gpkg', '*.geojson', '*.json', '*.kml', '*.gml']

def create_oriented_bounding_rectangles(input_folder, output_folder):
    files_to_process = []
    for ext in extensions:
        files_to_process.extend(glob.glob(os.path.join(input_folder, ext)))

    if not files_to_process:
        print("No supported vector files found in input folder.")
        return

    for file_path in files_to_process:
        file_name = os.path.basename(file_path)
        base_name, ext = os.path.splitext(file_name)
        ext = ext.lower()
        try:
            gdf = gpd.read_file(file_path)
        except:
            print(f"Failed to read {file_name}")
            continue

        if gdf.empty or gdf.geometry.is_empty.all():
            print(f"No valid geometries in {file_name}")
            continue

        obb_list = []
        for geom in gdf.geometry:
            if geom is None or geom.is_empty:
                continue
            try:
                obb = geom.minimum_rotated_rectangle
                obb_list.append(obb)
            except:
                continue

        if not obb_list:
            print(f"No bounding rectangles generated for {file_name}")
            continue

        obb_gdf = gpd.GeoDataFrame(geometry=obb_list, crs=gdf.crs)

        output_file_name = f"{base_name}_MBB{ext}"
        output_path = os.path.join(output_folder, output_file_name)

        driver = None
        if ext == ".shp":
            driver = "ESRI Shapefile"
        elif ext == ".gpkg":
            driver = "GPKG"
        elif ext in [".geojson", ".json"]:
            driver = "GeoJSON"
        elif ext == ".kml":
            driver = "KML"
        elif ext == ".gml":
            driver = "GML"

        try:
            obb_gdf.to_file(output_path, driver=driver)
            print(f"Oriented bounding rectangles saved to: {output_path}")
        except Exception as e:
            print(f"Error saving {file_name}: {e}")

create_oriented_bounding_rectangles(input_folder, output_folder)
print("All files processed!")


Oriented bounding rectangles saved to: /content/drive/My Drive/Shapefile_OMB/NL_buildings_obb.gpkg
Oriented bounding rectangles saved to: /content/drive/My Drive/Shapefile_OMB/NL_agriculture_obb.gpkg
Oriented bounding rectangles saved to: /content/drive/My Drive/Shapefile_OMB/NL_coastal_marine_infrastructure_obb.gpkg
Oriented bounding rectangles saved to: /content/drive/My Drive/Shapefile_OMB/NL_parking_obb.gpkg
Oriented bounding rectangles saved to: /content/drive/My Drive/Shapefile_OMB/NL_dams_obb.gpkg
Oriented bounding rectangles saved to: /content/drive/My Drive/Shapefile_OMB/NL_airports_obb.gpkg
Oriented bounding rectangles saved to: /content/drive/My Drive/Shapefile_OMB/NL_roads_obb.gpkg
Oriented bounding rectangles saved to: /content/drive/My Drive/Shapefile_OMB/NL_parks_obb.gpkg
All files processed!
